In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize

import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

2023-06-21 22:30:32.593063: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Inisialisasi pre-trained model BERT
from ABSA_SentimentMultiEmiten.model.bert import bert_ABSA
from ABSA_SentimentMultiEmiten.data.dataset import dataset_ABSA

DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(DEVICE)

pretrained_model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
model_ABSA = bert_ABSA(pretrained_model_name)
model_ABSA.to(DEVICE)

cuda:2


bert_ABSA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
          

In [3]:
# Load model
def load_model(model, path):
    model.load_state_dict(torch.load(path, map_location='cuda:2'), strict=False)
    return model

In [4]:
model_name = "indolem-indobert-gs/indolem-indobert-gs-1.pkl"
model_ABSA = load_model(model_ABSA, model_name)

In [5]:
# Function untuk embersihkan kalimat
def clean_sentences(sentences, aspect):
  new_sentences = []

  for sentence in sentences:
    if ((aspect in sentence)):
      index = sentence.index(aspect)
      emiten = (sentence[index-1].isalpha() == False) and (sentence[index+4].isalpha() == False) 
      if(emiten):
        new_sentences.append(sentence)
  return new_sentences

# Function untuk memisahkan kalimat menjadi beberapa token
def split_sentence(article):
  tokens = nltk.tokenize.sent_tokenize(article)
  return tokens

# Function untuk preprocessing teks input
def preprocessing_text(article, aspect):
  # Hapus URL
  if ('deviden' not in article) and ('dividen' not in article):
    article = re.sub('\S+.com|\S+.co.id|\S+.co|\S+.id', '.', article)
#   article = re.sub('\S+.com|\S+.co.id|\S+.co|\S+.id', '.', article)

  # Hapus titik yang bukan pemisah kalimat 
  article = article.replace("PT.", "PT").replace("Tbk.", "Tbk")
  for number in range(100):
    article = article.replace(" "+str(number)+". ", " " )

  # Ambil kalimat yang mengandung emiten
  sentences = split_sentence(article)
  arr_sentence_dirt = clean_sentences(sentences, aspect)

  arr_sentence_clean = []
  arr_sentence_clean.extend(arr_sentence_dirt)
  i = 0
  while (i < (len(arr_sentence_clean))):
    # Menghapus semua karakter selain huruf, angka, spasi, dan titik
    arr_sentence_clean[i] = re.sub(r'[^\w\d\s\.]+', '', arr_sentence_clean[i]) 
    
    # Mengubah kalimat menjadi huruf kecil
    arr_sentence_clean[i] = arr_sentence_clean[i].lower()
    
    # Menghapus angka
    arr_sentence_clean[i] = ''.join([x for x in arr_sentence_clean[i] if not x.isdigit()])
    i = i+1

  return arr_sentence_clean, arr_sentence_dirt 

In [6]:
# Prediksi kalimat
def predict(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs

# Controller predict
def predict_sentence(s, aspect):
  arr_sentence_clean, arr_sentence_dirt  = preprocessing_text(s, aspect)

  output = []
  i = 0
  sentiments = ["Negative", "Neutral", "Positive"]
  
  while (i < (len(arr_sentence_clean))):
    x, y, z = predict(arr_sentence_clean[i] , aspect, tokenizer)
    
    y_str = str(y)
    sentiment = sentiments[int(y_str[8])]
    output.append({
      "sentence": arr_sentence_dirt[i].strip(),
      "aspect": aspect,
      "sentiment": sentiment
    })
    i = i+1
  
  return output

# Mengambil data emiten
def get_data_emiten():
  return pd.read_csv("./data/daftar_emiten.csv")

# Prediksi artikel
def get_final_sentiment_artikel(artikel, data_emiten=[]):
  output = []

  if(data_emiten==[]):
    data_emiten = get_data_emiten().KodeEmiten
  else: 
    data_emiten = data_emiten.split()
  for emiten in data_emiten:
    if(emiten in artikel):
      output.extend(predict_sentence(artikel, emiten))
  
  return json.dumps(output, indent = 1)

In [7]:
teks = "Saham BBCA menguat 777 point menjadi 7.777 naik 10%, sedangkan saham BBRI mengalami netsell dengan nilai sebesar 10 Milyar rupiah, dan BBHI tidak terdapat pergerakan yang signifikan"

print(get_final_sentiment_artikel(teks))

[
 {
  "sentence": "Saham BBCA menguat 777 point menjadi 7.777 naik 10%, sedangkan saham BBRI mengalami netsell dengan nilai sebesar 10 Milyar rupiah, dan BBHI tidak terdapat pergerakan yang signifikan",
  "aspect": "BBCA",
  "sentiment": "Positive"
 },
 {
  "sentence": "Saham BBCA menguat 777 point menjadi 7.777 naik 10%, sedangkan saham BBRI mengalami netsell dengan nilai sebesar 10 Milyar rupiah, dan BBHI tidak terdapat pergerakan yang signifikan",
  "aspect": "BBHI",
  "sentiment": "Neutral"
 },
 {
  "sentence": "Saham BBCA menguat 777 point menjadi 7.777 naik 10%, sedangkan saham BBRI mengalami netsell dengan nilai sebesar 10 Milyar rupiah, dan BBHI tidak terdapat pergerakan yang signifikan",
  "aspect": "BBRI",
  "sentiment": "Negative"
 }
]


In [8]:
teks = """Ipotnews - Emiten BBHI dan LPPF tercatat menjadi jawara dalam jajaran top gainers di indeks Kompas100 pada Rabu (7/6) pagi.
Mengutip data IPOT 1 pukul 09.54 WIB, harga saham BBHI menguat 50 poin menjadi 1.125, atau naik 4,7%. Menyusul berikutnya adalah harga saham LPPF menguat 120 poin menjadi 3.410, atau naik 3,6%.
Sebagaimana diketahui, emiten bank digital, PT Allo Bank Indonesia Tbk (BBHI) berhasil meningkatkan performa kinerja pada kuartal I 2023 dengan peningkatan laba.
BBHI mencatatkan pertumbuhan laba bersih sepanjang tiga bulan pertama 2023 sebesar 21% secara tahunan (year-on-year/yoy) mencapai Rp90,49 miliar dari posisi sebelumnya Rp75 miliar.
Salah satu penopang pertumbuhan laba BBHI adalah lonjakan  pendapatan bunga hingga 204% secara tahunan menjadi Rp313,63 miliar pada kuartal I 2023 dari Rp103,3 miliar pada periode yang sama di tahun sebelumnya.
Namun sebaliknya BBHI juga mencatatkan peningkatan beban bunga sebesar 241% menjadi Rp76,55 miliar. Meskipun demikian, pendapatan bunga bersih bank tercatat tetap menebal 193% menjadi Rp237,08 miliar dibandingkan dengan kuartal I 2022 Rp80,83 miliar.
Selain itu, angka kerugian penurunan nilai aset keuangan ( impairment ) Allobank juga susut 70% menjadi Rp3,94 miliar hingga Maret 2023, dari Rp13,21 miliar pada akhir Maret 2022.
Sementara itu, pada kuartal I 2023, PT Matahari Department Store Tbk (LPPF) memperluas bisnisnya dengan membuka dua gerai baru di Balikpapan, Kalimantan Timur dan Kuta, Bali. Pembukaan gerai baru di Plaza Balikpapan dilakukan pada Rabu (05/04/2023) dan di Discovery Shopping Mall, Kuta, pada 6 April 2023.
"Dengan membawa tema 'Time To Grow' di tahun ini, kami membuka gerai baru keenam dan ketujuh dari program pembukaan gerai yang sudah direncanakan perseroan. Melalui dua gerai baru ini, kami ingin menghadirkan berbagai pilihan dan pengalaman berbelanja, serta menjadikan Matahari sebagai pilihan utama berbelanja di musim Lebaran ini," kata CEO Matahari Terry O'Connor dalam keterangan resminya, Rabu (5/4/2023).
Dengan konsep baru yang lebih segar, menurut Terry, kedua toko ini menawarkan merchandise yang diperbarui dan pengalaman berbelanja pelanggan yang lebih baik. Matahari menggunakan sistem pencahayaan 100% LED sebagai komitmen perusahaan terhadap lingkungan.
"Kami percaya, kehadiran dua gerai dengan karakteristik modern ini akan menghadirkan pilihan belanja yang berbeda bagi masyarakat setempat," tutur dia.
"""

print(get_final_sentiment_artikel(teks))

[
 {
  "sentence": "Ipotnews - Emiten BBHI dan LPPF tercatat menjadi jawara dalam jajaran top gainers di indeks Kompas100 pada Rabu (7/6) pagi.",
  "aspect": "BBHI",
  "sentiment": "Positive"
 },
 {
  "sentence": "Mengutip data IPOT 1 pukul 09.54 WIB, harga saham BBHI menguat 50 poin menjadi 1.125, atau naik 4,7%.",
  "aspect": "BBHI",
  "sentiment": "Positive"
 },
 {
  "sentence": "Sebagaimana diketahui, emiten bank digital, PT Allo Bank Indonesia Tbk (BBHI) berhasil meningkatkan performa kinerja pada kuartal I 2023 dengan peningkatan laba.",
  "aspect": "BBHI",
  "sentiment": "Positive"
 },
 {
  "sentence": "BBHI mencatatkan pertumbuhan laba bersih sepanjang tiga bulan pertama 2023 sebesar 21% secara tahunan (year-on-year/yoy) mencapai Rp90,49 miliar dari posisi sebelumnya Rp75 miliar.",
  "aspect": "BBHI",
  "sentiment": "Positive"
 },
 {
  "sentence": "Salah satu penopang pertumbuhan laba BBHI adalah lonjakan  pendapatan bunga hingga 204% secara tahunan menjadi Rp313,63 miliar pada

In [9]:
teks = """Ipotnews - PT Provident Investasi Bersama Tbk (PALM) akan melakukan buyback saham sebanyak-banyaknya 103.950.000 saham atau 1,46% dari seluruh modal ditempatkan dan disetor penuh.
Periode buyback saham mulai dilaksanakan dari 21 Juni 2023 sampai dengan 20 Juni 2024. Anggaran yang disiapkan perseroan untuk aksi korporasi ini sebesar Rp80,66 miliar.
Presiden Direktur PALM, Tri Boewono mengungkapkan buyback saham tersebut dapat memberikan fleksibilitas yang lebih besar kepada perseroan dalam mengelola modal untuk mencapai struktur permodalan yang efisien untuk meningkatkan nilai pemegang saham.
"Kami berkeyakinan bahwa buyback tidak akan memberikan dampak negatif yang material terhadap kegiatan usaha Perseroan. Perseroan memiliki modal kerja dan arus kas (cash flow) yang memadai untuk melaksanakan pembiayaan transaksi bersamaan dengan kegiatan usaha Perseroan," ucapTrei Boewono dalam keterangannya, Rabu (21/6).
Keputusan untuk melakukan buyback ini telah mendapat restu dari pemegang saham saat pelaksanaan RUPST pada Rabu (21/6).
Dalam RUPST ini pemegang saham juga menyetujui pengangkatan Ellen Kartika sebagai Direktur Investasi dan Portofolio menggantikan posisi Devin Antonio Ridwan, yang kini menjabat sebagai Presiden Direktur PT Merdeka Battery Materials Tbk (MBMA), anak perusahaan PT Merdeka Copper Gold Tbk (MDKA). Perubahan komposisi ini akan mendukung strategi perseroan dalam mencari dan mengeksekusi peluang investasi di masa mendatang demi pertumbuhan kinerja Perseroan.
"Kami mengucapkan selamat kepada Ibu Ellen Kartika dan terima kasih atas kontribusi Bapak Devin Antonio Ridwan yang berperan besar dalam memajukan Perseroan hingga sampai pada posisi saat ini. Kami optimistis, dengan pengalaman panjang Ibu Ellen Kartika dalam investasi dan portofolio, akan juga membawa perubahan yang berkelanjutan bagi kinerja Perseroan," kata Tri Boewono.(Marjudin)
Sumber : admin"""

print(get_final_sentiment_artikel(teks))

[
 {
  "sentence": "Dalam RUPST ini pemegang saham juga menyetujui pengangkatan Ellen Kartika sebagai Direktur Investasi dan Portofolio menggantikan posisi Devin Antonio Ridwan, yang kini menjabat sebagai .en Direktur PT Merdeka Battery Materials Tbk (MBMA), anak perusahaan PT Merdeka Copper Gold Tbk (MDKA).",
  "aspect": "MDKA",
  "sentiment": "Neutral"
 },
 {
  "sentence": "Ipotnews - PT .ent Investasi Bersama Tbk (PALM) akan melakukan buyback saham sebanyak-banyaknya 103.950.000 saham atau 1,46% dari seluruh modal ditempatkan dan disetor penuh.",
  "aspect": "PALM",
  "sentiment": "Positive"
 },
 {
  "sentence": ".en Direktur PALM, Tri Boewono mengungkapkan buyback saham tersebut dapat memberikan fleksibilitas yang lebih besar kepada perseroan dalam mengelola modal untuk mencapai struktur permodalan yang efisien untuk meningkatkan nilai pemegang saham.",
  "aspect": "PALM",
  "sentiment": "Positive"
 }
]


In [ ]:
torch.cuda.empty_cache()